# **Modelamiento**

## Librerías

In [ ]:
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3 MB 1.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 8.5 MB/s 


In [ ]:
#Generales
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Proceso de Modelación
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error,r2_score,median_absolute_error, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, ShuffleSplit

## Lectura Información

In [ ]:
#Descargar datasets desde github
!git clone https://github.com/andres-soto-h/monografia-udea-eacd.git

Cloning into 'monografia-udea-eacd'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [ ]:
#Lectura del dataset transformado
df_propiedades=pd.read_csv('/content/monografia-udea-eacd/df_prop_clean_12082021.csv', delimiter=';', encoding='latin1')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd '/content/drive/My Drive/Colab Notebooks/Seminario/DATASETS'
# # !ls
# df_propiedades=pd.read_csv('df_prop_clean_12082021.csv', delimiter=';', encoding='latin1') 
# print(df_propiedades.shape)

In [ ]:
df_propiedades.rename(columns={"municipio":"ubicacion"}, inplace=True)

**NOTA:** se decide eliminar la información del municipio de **La Unión**.

In [ ]:
mask_union = df_propiedades['ubicacion']!='la uniÃ³n'
df_propiedades = df_propiedades[mask_union]

In [ ]:
df_propiedades['ubicacion'].value_counts()

rionegro                  1241
el retiro                  439
la ceja                    342
guarne                     196
marinilla                  117
el carmen de viboral        62
san antonio de pereira      38
llanogrande                 31
santuario                   15
Name: ubicacion, dtype: int64

**Nota2:** Se convierte la variable Estrato en ordinal

In [ ]:
df_propiedades['estrato'] = df_propiedades['estrato'].replace('Campestre', '0')
df_propiedades['estrato'] = df_propiedades['estrato'].astype(int)
df_propiedades['estrato'].value_counts()

4    947
5    636
3    584
0    162
6     89
2     58
1      5
Name: estrato, dtype: int64

## **Ajuste del Modelo**

**Funciones**

In [ ]:
#Métricas datos transformados
def median_absolute_error2(y_true, y_pred):
  return median_absolute_error(np.expm1(y_true), np.expm1(y_pred))

def mean_squared_error2(y_true, y_pred):
  return mean_squared_error(np.expm1(y_true), np.expm1(y_pred), squared=False)


def mean_absolute_percentage_error2(y_true, y_pred):
  return mean_absolute_percentage_error(np.expm1(y_true), np.expm1(y_pred))
  
def r2_score2(y_true, y_pred):
  return r2_score(np.expm1(y_true), np.expm1(y_pred))

In [ ]:
def div_train_test(X, y, var_stratify = ''):
  
  if var_stratify != '': 
    x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.9, stratify = X[var_stratify],  random_state = 42)

    var_stratify_train = x_train[var_stratify]
    #ubicacion_test = X_test['ubicacion']

    x_train.drop(var_stratify,axis=1, inplace=True)
    x_test.drop(var_stratify,axis=1, inplace=True)

    #Variable Objetivo Transformada
    y_train_t = np.log1p(y_train)
    y_test_t = np.log1p(y_test)

    return x_train, x_test, y_train, y_test, y_train_t, y_test_t, var_stratify_train

  else:
    x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.9,  random_state = 42)

    #Variable Objetivo Transformada
    y_train_t = np.log1p(y_train)
    y_test_t = np.log1p(y_test)
  
  return x_train, x_test, y_train, y_test, y_train_t, y_test_t

def busqueda_hiperparametros(x_train, y_train, var_stratify_train, estimator,param_grid = [] , scoring_med = 'neg_median_absolute_error', transf=False):

  if len(var_stratify_train) != 0:
    
    print('Seccion Stratify')
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)
    stratified = sss.split(x_train, var_stratify_train)
    
    if transf: 
      print('**para datos transformados**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error2), 'r2':make_scorer(r2_score2) , 'rmse':make_scorer(mean_squared_error2), 'mape':make_scorer(mean_absolute_percentage_error2)}
    else:
      print('**para datos sin transformar**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error), 'r2':make_scorer(r2_score) , 'rmse':make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
    
    grid = GridSearchCV(estimator, param_grid=param_grid, cv = stratified, scoring = scoring_grid , refit = 'metrica',  return_train_score=True, n_jobs=-1, verbose=8)
    return grid.fit(x_train, y_train)
  
  else:
    print('OutStratify')
    
    if transf: 
      print('**para datos transformados**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error2), 'r2':make_scorer(r2_score2) , 'rmse':make_scorer(mean_squared_error2), 'mape':make_scorer(mean_absolute_percentage_error2)}
    else:
      print('**para datos sin transformar**')
      scoring_grid={'metrica':scoring_med,'meae':make_scorer(median_absolute_error), 'r2':make_scorer(r2_score) , 'rmse':make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
    
    grid = GridSearchCV(estimator, param_grid=param_grid, scoring = scoring_grid, refit = 'metrica', return_train_score=True, n_jobs=-1, verbose=8 )
    return grid.fit(X_train, y_train)

def metricas(model,y_train, p_train, y_test, p_test): 

  resultados = {
      'parametros' : [model.get_params()['steps'][1][1]],
      'MeAE_train' : [median_absolute_error(y_train,p_train)],
      'MeAE_test'  : [median_absolute_error(y_test, p_test)],
      'MAPE_train' : [mean_absolute_percentage_error(y_train,p_train)],
      'MAPE_test'  : [mean_absolute_percentage_error(y_test, p_test)],
      'r2_train'   : [r2_score(y_train,p_train)],
      'r2_test'    : [r2_score(y_test, p_test)],
      'rmse_train' : [mean_squared_error(y_train,p_train,squared = False)],
      'rmse_test'  : [mean_squared_error(y_test,p_test,squared = False)]
  }
  return pd.DataFrame(resultados)

**División Covariables  y Variable Objetivo**

In [ ]:
data_model=df_propiedades.copy()
#columnas_quitar=['tipo','url','titulo','descripcion','caractint','caractext','caractsec']
columnas_quitar=['url','titulo','descripcion','caractint','caractext','caractsec','otros_datos']
data_model.drop(columnas_quitar, axis=1, inplace=True)
data_model.shape

(2481, 177)

In [ ]:
X = data_model.drop(['precio'], axis=1)
y = data_model['precio']

**OneHotEncoder Variables Categóricas**

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
data_aux = pd.DataFrame(enc.fit_transform(X[['tipo','tipo_propiedad','ubicacion','antiguedad']]).toarray(), columns= enc.get_feature_names(['tipo','tipo_propiedad','ubicacion','antiguedad']))
data_aux['fila'] = range(0, X.shape[0])
data_aux.head()

X.drop(['tipo','tipo_propiedad','antiguedad'],axis=1,inplace=True)
X['fila'] = range(0, X.shape[0])

X = pd.merge(data_aux,X, on = 'fila', how='inner')
X.drop(['fila'], axis=1, inplace=True)
X.shape

(2481, 194)

**División train y Test**

In [ ]:
X_train, X_test, Y_train, Y_test, Y_train_t, Y_test_t, var_stratify_train = div_train_test(X, y, var_stratify = 'ubicacion')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
hasta aquí

### **Random Forest**

In [ ]:
pipe = Pipeline(steps = [('scaler', StandardScaler()), ('rf', RandomForestRegressor(random_state=42))])

####**Sin transformar la Y**

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'rf__n_estimators':[80,100,120,150], 'rf__max_depth':[3,5,10,20,25], 'rf__min_samples_split':[2,3,5,10,15],'rf__min_samples_leaf':[1,2,5,8]}
para_grid = {'rf__n_estimators':[100,120,150], 'rf__max_depth':[6,7,8,9], 'rf__min_samples_split':[4,5,6], 'rf__min_samples_leaf':[2,3,4]}

modelo_rf = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe, param_grid = para_grid, transf=False)
modelo_rf

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=<generator object BaseShuffleSplit.split at 0x7f55756b4550>,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [6, 7, 8, 9],
                         'rf__min_samples_leaf': [2, 3, 4],
                         'rf__min_samples_split': [4, 5, 6],
                         'rf__n_estimators': [100, 120, 150]},
             refit='metrica', return_train_score=True,
             scoring={'mape': make_scorer(mean_absolute_percentage_error),
                      'meae': make_scorer(median_absolute_error),
                      'metrica': 'neg_median_absolute_error',
                      'r2': make_scorer(r2_score),
                      'rmse': make_scorer(mean_squared_error)},
             verbose=8)

In [ ]:
metricas_rf = metricas(modelo_rf.best_estimator_,Y_train, modelo_rf.predict(X_train), Y_test, modelo_rf.predict(X_test))
metricas_rf

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.348362e+07,7.340018e+07,0.163045,0.220709,0.893049,0.771647,1.845402e+08,3.046399e+08


In [ ]:
modelo_rf.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestRegressor(max_depth=9, min_samples_leaf=4,
                                       min_samples_split=4, n_estimators=150,
                                       random_state=42))])

In [ ]:
dd=pd.DataFrame(modelo_rf.cv_results_)
dd[dd['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica',
'mean_test_meae', 'std_test_meae', 'rank_test_meae','mean_train_meae','std_train_meae',
'mean_test_r2', 'std_test_r2','rank_test_r2','mean_train_r2', 'std_train_r2',
'mean_test_rmse','std_test_rmse', 'rank_test_rmse','mean_train_rmse', 'std_train_rmse',
'mean_test_mape','std_test_mape', 'rank_test_mape', 'mean_train_mape', 'std_train_mape']]

,params,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_test_meae,std_test_meae,rank_test_meae,mean_train_meae,std_train_meae,mean_test_r2,std_test_r2,rank_test_r2,mean_train_r2,std_train_r2,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_train_rmse,std_train_rmse,mean_test_mape,std_test_mape,rank_test_mape,mean_train_mape,std_train_mape
101,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 4...",-7.123932e+07,7.119693e+06,-5.316468e+07,712722.460535,7.123932e+07,7.119693e+06,106,5.316468e+07,712722.460535,0.729726,0.034436,24,0.89631,0.001892,9.178554e+16,1.145901e+16,84,3.272395e+16,6.144098e+14,0.230457,0.010604,88,0.159293,0.001785
104,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 4...",-7.123932e+07,7.119693e+06,-5.316468e+07,712722.460535,7.123932e+07,7.119693e+06,106,5.316468e+07,712722.460535,0.729726,0.034436,24,0.89631,0.001892,9.178554e+16,1.145901e+16,84,3.272395e+16,6.144098e+14,0.230457,0.010604,88,0.159293,0.001785
107,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 4...",-7.123932e+07,7.119693e+06,-5.316468e+07,712722.460535,7.123932e+07,7.119693e+06,106,5.316468e+07,712722.460535,0.729726,0.034436,24,0.89631,0.001892,9.178554e+16,1.145901e+16,84,3.272395e+16,6.144098e+14,0.230457,0.010604,88,0.159293,0.001785


####**Usando Y transformada**

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'rf__n_estimators':[80,100,120,150], 'rf__max_depth':[3,5,10,20,25], 'rf__min_samples_split':[2,3,5,10,15],'rf__min_samples_leaf':[1,2,5,8]}
para_grid = {'rf__n_estimators':[100,120,150], 'rf__max_depth':[6,7,8,9], 'rf__min_samples_split':[4,5,6], 'rf__min_samples_leaf':[2,3,4]}

modelo_rf_t = busqueda_hiperparametros(X_train, Y_train_t,var_stratify_train, pipe, param_grid = para_grid, transf = True)
modelo_rf_t

Seccion Stratify
**para datos transformados**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=<generator object BaseShuffleSplit.split at 0x7f55755eccd0>,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [6, 7, 8, 9],
                         'rf__min_samples_leaf': [2, 3, 4],
                         'rf__min_samples_split': [4, 5, 6],
                         'rf__n_estimators': [100, 120, 150]},
             refit='metrica', return_train_score=True,
             scoring={'mape': make_scorer(mean_absolute_percentage_error2),
                      'meae': make_scorer(median_absolute_error2),
                      'metrica': 'neg_median_absolute_error',
                      'r2': make_scorer(r2_score2),
                      'rmse': make_scorer(mean_squared_error2)},
             verbose=8)

In [ ]:
metricas_rf_t = metricas(modelo_rf_t.best_estimator_,Y_train, np.expm1(modelo_rf_t.predict(X_train)), Y_test, np.expm1(modelo_rf_t.predict(X_test)))
metricas_rf_t

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"(DecisionTreeRegressor(max_depth=9, max_featur...",4.630985e+07,7.502619e+07,0.135886,0.205783,0.886589,0.743525,1.900325e+08,3.228539e+08


In [ ]:
modelo_rf_t.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestRegressor(max_depth=9, min_samples_leaf=2,
                                       min_samples_split=5, n_estimators=120,
                                       random_state=42))])

In [ ]:
dd_t=pd.DataFrame(modelo_rf_t.cv_results_)
dd_t[dd_t['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica',
'mean_test_meae', 'std_test_meae', 'rank_test_meae','mean_train_meae','std_train_meae',
'mean_test_r2', 'std_test_r2','rank_test_r2','mean_train_r2', 'std_train_r2',
'mean_test_rmse','std_test_rmse', 'rank_test_rmse','mean_train_rmse', 'std_train_rmse',
'mean_test_mape','std_test_mape', 'rank_test_mape', 'mean_train_mape', 'std_train_mape']]

,params,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_test_meae,std_test_meae,rank_test_meae,mean_train_meae,std_train_meae,mean_test_r2,std_test_r2,rank_test_r2,mean_train_r2,std_train_r2,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_train_rmse,std_train_rmse,mean_test_mape,std_test_mape,rank_test_mape,mean_train_mape,std_train_mape
85,"{'rf__max_depth': 9, 'rf__min_samples_leaf': 2...",-0.150624,0.014653,-0.094544,0.001878,7.209165e+07,1.092661e+07,92,4.453310e+07,711963.505776,0.703439,0.032058,13,0.890133,0.002761,3.171866e+08,2.045609e+07,95,1.861942e+08,2.566814e+06,0.213649,0.010387,100,0.132163,0.001725


### **Gradient Boosting Regression**

####**Sin tranformar la Y**

In [ ]:
pipe_gbt = Pipeline(steps = [('scaler', StandardScaler()), ('gbt', GradientBoostingRegressor(random_state=42))])

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'gbt__n_estimators':[70,90,100,120,180], 'gbt__max_depth':[3,5,10,12,15,20,25], 'gbt__min_samples_split':[5,10,15,17,25,30,40], 'gbt__min_samples_leaf':[1,3,5]}
para_grid = {'gbt__n_estimators':[100,120,150], 'gbt__max_depth':[6,7,8,9], 'gbt__min_samples_split':[4,5,6], 'gbt__min_samples_leaf':[2,3,4]}

modelo_gbt = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe_gbt, param_grid = para_grid )

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
metricas_gbt = metricas(modelo_gbt.best_estimator_,Y_train, modelo_gbt.predict(X_train), Y_test, modelo_gbt.predict(X_test))
metricas_gbt

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,([DecisionTreeRegressor(criterion='friedman_ms...,7.774991e+06,7.417244e+07,0.02737,0.206195,0.998624,0.785648,2.092951e+07,2.951534e+08


In [ ]:
modelo_gbt.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('gbt',
                 GradientBoostingRegressor(max_depth=9, min_samples_leaf=3,
                                           min_samples_split=4,
                                           n_estimators=150,
                                           random_state=42))])

####**Usando Y transformada**

**Búsqueda de hiperparámetros**

In [ ]:
# para_grid = {'gbt__n_estimators':[70,90,100,120,180], 'gbt__max_depth':[3,5,10,12,15,20,25], 'gbt__min_samples_split':[5,10,15,17,25,30,40], 'gbt__min_samples_leaf':[1,3,5]}
para_grid = {'gbt__n_estimators':[100,120,150], 'gbt__max_depth':[6,7,8,9], 'gbt__min_samples_split':[4,5,6], 'gbt__min_samples_leaf':[2,3,4]}
modelo_gbt_t = busqueda_hiperparametros(X_train, Y_train_t,var_stratify_train, pipe_gbt, param_grid = para_grid, transf=True )

Seccion Stratify
**para datos transformados**
Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
metricas_gbt_t = metricas(modelo_gbt_t.best_estimator_,Y_train, np.expm1(modelo_gbt_t.predict(X_train)), Y_test, np.expm1(modelo_gbt_t.predict(X_test)))
metricas_gbt_t

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,([DecisionTreeRegressor(criterion='friedman_ms...,9.519395e+06,7.527124e+07,0.028575,0.189862,0.994003,0.790916,4.369992e+07,2.915036e+08


In [ ]:
modelo_gbt_t.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('gbt',
                 GradientBoostingRegressor(max_depth=9, min_samples_leaf=4,
                                           min_samples_split=4,
                                           n_estimators=120,
                                           random_state=42))])

### **XG Boost**

#### Sin transformar la Y

In [ ]:
import xgboost as xgb
pipe_xgb = Pipeline(steps = [
    ('scaler', StandardScaler()), 
    ('xgb',  xgb.XGBRegressor(
        objective='reg:squarederror', n_jobs=-1, 
    ))
])

**Búsqueda de Hiperparámetros**

In [ ]:
para_grid = {'xgb__n_estimators':[100,120,150], 'xgb__max_depth':[5,6], 'xgb__learning_rate':[0.05,0.1], 'xgb__reg_alpha':[0.01,0.5,0.1]}
modelo_xgb = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe_xgb, param_grid = para_grid )

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
metricas_xgb = metricas(modelo_xgb.best_estimator_,Y_train, modelo_xgb.predict(X_train), Y_test, modelo_xgb.predict(X_test))
metricas_xgb

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"XGBRegressor(max_depth=6, n_jobs=-1, objective...",43208272.0,85208512.0,0.118753,0.21386,0.961429,0.783696,1.108224e+08,2.964941e+08


In [ ]:
modelo_xgb.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb',
                 XGBRegressor(max_depth=6, n_jobs=-1,
                              objective='reg:squarederror', reg_alpha=0.01))])

### **Red Neuronal**

####**Sin tranformar la Y**

In [ ]:
from sklearn.neural_network import MLPRegressor
pipe_rnn = Pipeline(steps = [('scaler', StandardScaler()), ('rnn', MLPRegressor(activation='relu',max_iter=100,random_state=42))])

**Búsqueda de Hiperparámetros**

In [ ]:
# para_grid = {'rnn__hidden_layer_sizes':[(45,25),(90,45),(95,50),(110,60)],'rnn__alpha':[0.001,0.01,0.1]}

# para_grid = {'rnn__hidden_layer_sizes': [(45,25),(95,50),(110,60)],
#     'rnn__alpha': np.logspace(-3, 3, 10),
#     'rnn__learning_rate_init': [0.001, 0.01,0.1,0.2],
#     'rnn__solver':['adam']}

para_grid = {'rnn__hidden_layer_sizes': [(45,25),(95,50),(110,60),(150,75),(180,90)],
    'rnn__alpha': [0.001,0.004,0.01,0.1],
    'rnn__learning_rate_init': [0.001,0.004,0.01,0.1],
    'rnn__max_iter':[200,500]}

modelo_rnn = busqueda_hiperparametros(X_train, Y_train,var_stratify_train, pipe_rnn, param_grid = para_grid,transf=False )

Seccion Stratify
**para datos sin transformar**
Fitting 5 folds for each of 160 candidates, totalling 800 fits


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
metricas_rnn = metricas(modelo_rnn.best_estimator_,Y_train, modelo_rnn.predict(X_train), Y_test, modelo_rnn.predict(X_test))
metricas_rnn

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"MLPRegressor(alpha=0.001, hidden_layer_sizes=(...",8.179092e+07,1.592290e+08,0.240227,0.359388,0.812771,0.604068,2.441666e+08,4.011382e+08


In [ ]:
modelo_rnn.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rnn',
                 MLPRegressor(alpha=0.001, hidden_layer_sizes=(45, 25),
                              learning_rate_init=0.1, max_iter=500,
                              random_state=42))])

####**Usando Y transformada**

In [ ]:
# para_grid = {'rnn__hidden_layer_sizes': [(45,25),(95,50),(110,60),(150,75),(180,90)],
#     'rnn__alpha': np.logspace(-3, 3, 10),
#     'rnn__learning_rate_init': [0.001, 0.01]}
para_grid = {'rnn__hidden_layer_sizes': [(45,25),(95,50),(110,60),(150,75),(180,90)],
    'rnn__alpha': [0.001,0.004,0.01,0.1],
    'rnn__learning_rate_init': [0.001,0.004,0.01,0.1],
    'rnn__max_iter':[200,500]}
    
modelo_rnn_t = busqueda_hiperparametros(X_train, Y_train_t,var_stratify_train, pipe_rnn, param_grid = para_grid, transf=True )

Seccion Stratify
**para datos transformados**
Fitting 5 folds for each of 160 candidates, totalling 800 fits


In [ ]:
metricas_rnn_t = metricas(modelo_rnn_t.best_estimator_,Y_train, np.expm1(modelo_rnn_t.predict(X_train)), Y_test, np.expm1(modelo_rnn_t.predict(X_test)))
metricas_rnn_t

,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,"MLPRegressor(alpha=0.1, hidden_layer_sizes=(18...",2.008935e+07,2.186861e+08,0.067313,135263.763282,0.966192,-3.872213e+13,1.037551e+08,3.967011e+15


In [ ]:
modelo_rnn_t.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rnn',
                 MLPRegressor(alpha=0.1, hidden_layer_sizes=(180, 90),
                              random_state=42))])

Referencia: https://www.cienciadedatos.net/documentos/py35-redes-neuronales-python.html

In [ ]:
resultados_rnn =  pd.concat([metricas_rnn, metricas_rnn_t],ignore_index= True)
metricas_rnn.to_csv('resultados_train_test_general_rnn.csv',sep=";",decimal=",")

KNN con Keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_shape=(input_dim,)),
    Activation('tanh'),
    Dense(nb_classes),
    Activation('softmax'),
])

## Resumen

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/Seminario/Etapa Modelamiento/Resultados'

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks/Seminario/Etapa Modelamiento/Resultados


In [ ]:
import os
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Seminario/Etapa Modelamiento/Resultados'

In [ ]:
cross_validation_rf = pd.DataFrame(modelo_rf.cv_results_)
cross_validation_rf = cross_validation_rf[cross_validation_rf['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_rf['escenario']='modelo_rf'

cross_validation_rft=pd.DataFrame(modelo_rf_t.cv_results_)
cross_validation_rft = cross_validation_rft[cross_validation_rft['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_rft['escenario']='modelo_rft'

cross_validation_gbt=pd.DataFrame(modelo_gbt.cv_results_)
cross_validation_gbt = cross_validation_gbt[cross_validation_gbt['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_gbt['escenario']='modelo_gbt'

cross_validation_gbtt=pd.DataFrame(modelo_gbt_t.cv_results_)
cross_validation_gbtt = cross_validation_gbtt[cross_validation_gbtt['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_gbtt['escenario']='modelo_gbtt'

cross_validation_rnn=pd.DataFrame(modelo_rnn.cv_results_)
cross_validation_rnn = cross_validation_rnn[cross_validation_rnn['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_rnn['escenario']='modelo_rnn'

cross_validation_rnnt=pd.DataFrame(modelo_rnn_t.cv_results_)
cross_validation_rnnt = cross_validation_rnnt[cross_validation_rnnt['rank_test_metrica']==1][['params','mean_test_metrica', 'std_test_metrica','mean_train_metrica', 'std_train_metrica','mean_train_meae','mean_test_meae','mean_train_mape','mean_test_mape','mean_train_r2','mean_test_r2','mean_train_rmse','mean_test_rmse']]
cross_validation_rnnt['escenario']='modelo_rnnt'

resultado_cross = pd.concat([cross_validation_rf,cross_validation_rft, cross_validation_gbt, cross_validation_gbtt,cross_validation_rnn,cross_validation_rnnt],
                            ignore_index = True)

columna = pd.DataFrame(resultado_cross['params'])

resultado_cross.drop(['params'],axis=1, inplace=True)
resultado_cross = resultado_cross.drop_duplicates()
resultado_cross.reset_index(inplace=True, drop=False)
resultado_cross

,index,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_train_meae,mean_test_meae,mean_train_mape,mean_test_mape,mean_train_r2,mean_test_r2,mean_train_rmse,mean_test_rmse,escenario
0,0,-7.123932e+07,7.119693e+06,-5.316468e+07,7.127225e+05,5.316468e+07,7.123932e+07,0.159293,2.304569e-01,0.896310,7.297261e-01,3.272395e+16,9.178554e+16,modelo_rf
1,3,-1.506244e-01,1.465277e-02,-9.454374e-02,1.877897e-03,4.453310e+07,7.209165e+07,0.132163,2.136494e-01,0.890133,7.034389e-01,1.861942e+08,3.171866e+08,modelo_rft
2,4,-6.142328e+07,7.260291e+06,-6.570505e+06,8.794403e+05,6.570505e+06,6.142328e+07,0.024161,2.148322e-01,0.998964,7.172251e-01,3.263657e+14,9.634683e+16,modelo_gbt
3,7,-1.298900e-01,1.458501e-02,-1.590032e-02,1.141182e-03,8.282135e+06,6.096663e+07,0.025676,1.989429e-01,0.995258,7.154888e-01,3.866727e+07,3.104061e+08,modelo_gbtt
4,10,-1.278012e+08,8.317124e+06,-9.215281e+07,3.327533e+06,9.215281e+07,1.278012e+08,0.261387,3.495676e-01,0.790765,5.990284e-01,6.604700e+16,1.372560e+17,modelo_rnn
5,11,-4.479736e-01,6.230440e-02,-8.669250e-02,1.315596e-02,4.284584e+07,1.957777e+08,0.155250,9.538777e+12,0.241742,-7.872653e+29,4.136689e+08,2.426977e+23,modelo_rnnt


In [ ]:
resultado_metricas = pd.concat([metricas_rf, metricas_rf_t, metricas_gbt, metricas_gbt_t, metricas_rnn, metricas_rnn_t],
                            ignore_index = True)
resultado_metricas.drop_duplicates(inplace=True)
resultado_metricas.reset_index(inplace=True, drop=False)
resultado_metricas

,index,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test
0,0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.348362e+07,7.340018e+07,0.163045,0.220709,0.893049,7.716474e-01,1.845402e+08,3.046399e+08
1,1,"(DecisionTreeRegressor(max_depth=9, max_featur...",4.630985e+07,7.502619e+07,0.135886,0.205783,0.886589,7.435254e-01,1.900325e+08,3.228539e+08
2,2,([DecisionTreeRegressor(criterion='friedman_ms...,7.774991e+06,7.417244e+07,0.027370,0.206195,0.998624,7.856478e-01,2.092951e+07,2.951534e+08
3,3,([DecisionTreeRegressor(criterion='friedman_ms...,9.519395e+06,7.527124e+07,0.028575,0.189862,0.994003,7.909163e-01,4.369992e+07,2.915036e+08
4,4,"MLPRegressor(alpha=0.001, hidden_layer_sizes=(...",8.179092e+07,1.592290e+08,0.240227,0.359388,0.812771,6.040681e-01,2.441666e+08,4.011382e+08
5,5,"MLPRegressor(alpha=0.1, hidden_layer_sizes=(18...",2.008935e+07,2.186861e+08,0.067313,135263.763282,0.966192,-3.872213e+13,1.037551e+08,3.967011e+15


In [ ]:
resultados =  pd.concat([resultado_metricas, resultado_cross], axis=1)
resultados

,index,parametros,MeAE_train,MeAE_test,MAPE_train,MAPE_test,r2_train,r2_test,rmse_train,rmse_test,index,mean_test_metrica,std_test_metrica,mean_train_metrica,std_train_metrica,mean_train_meae,mean_test_meae,mean_train_mape,mean_test_mape,mean_train_r2,mean_test_r2,mean_train_rmse,mean_test_rmse,escenario
0,0,"(DecisionTreeRegressor(max_depth=9, max_featur...",5.348362e+07,7.340018e+07,0.163045,0.220709,0.893049,7.716474e-01,1.845402e+08,3.046399e+08,0,-7.123932e+07,7.119693e+06,-5.316468e+07,7.127225e+05,5.316468e+07,7.123932e+07,0.159293,2.304569e-01,0.896310,7.297261e-01,3.272395e+16,9.178554e+16,modelo_rf
1,1,"(DecisionTreeRegressor(max_depth=9, max_featur...",4.630985e+07,7.502619e+07,0.135886,0.205783,0.886589,7.435254e-01,1.900325e+08,3.228539e+08,3,-1.506244e-01,1.465277e-02,-9.454374e-02,1.877897e-03,4.453310e+07,7.209165e+07,0.132163,2.136494e-01,0.890133,7.034389e-01,1.861942e+08,3.171866e+08,modelo_rft
2,2,([DecisionTreeRegressor(criterion='friedman_ms...,7.774991e+06,7.417244e+07,0.027370,0.206195,0.998624,7.856478e-01,2.092951e+07,2.951534e+08,4,-6.142328e+07,7.260291e+06,-6.570505e+06,8.794403e+05,6.570505e+06,6.142328e+07,0.024161,2.148322e-01,0.998964,7.172251e-01,3.263657e+14,9.634683e+16,modelo_gbt
3,3,([DecisionTreeRegressor(criterion='friedman_ms...,9.519395e+06,7.527124e+07,0.028575,0.189862,0.994003,7.909163e-01,4.369992e+07,2.915036e+08,7,-1.298900e-01,1.458501e-02,-1.590032e-02,1.141182e-03,8.282135e+06,6.096663e+07,0.025676,1.989429e-01,0.995258,7.154888e-01,3.866727e+07,3.104061e+08,modelo_gbtt
4,4,"MLPRegressor(alpha=0.001, hidden_layer_sizes=(...",8.179092e+07,1.592290e+08,0.240227,0.359388,0.812771,6.040681e-01,2.441666e+08,4.011382e+08,10,-1.278012e+08,8.317124e+06,-9.215281e+07,3.327533e+06,9.215281e+07,1.278012e+08,0.261387,3.495676e-01,0.790765,5.990284e-01,6.604700e+16,1.372560e+17,modelo_rnn
5,5,"MLPRegressor(alpha=0.1, hidden_layer_sizes=(18...",2.008935e+07,2.186861e+08,0.067313,135263.763282,0.966192,-3.872213e+13,1.037551e+08,3.967011e+15,11,-4.479736e-01,6.230440e-02,-8.669250e-02,1.315596e-02,4.284584e+07,1.957777e+08,0.155250,9.538777e+12,0.241742,-7.872653e+29,4.136689e+08,2.426977e+23,modelo_rnnt


In [ ]:
resultados.to_csv('resultados_train_test_general.csv',sep=";",decimal=",")

In [ ]:
cross_validation_rf = pd.DataFrame(modelo_rf.cv_results_)
cross_validation_rf['escenario']='modelo_rf'

cross_validation_rft=pd.DataFrame(modelo_rf_t.cv_results_)
cross_validation_rft['escenario']='modelo_rft'

cross_validation_gbt=pd.DataFrame(modelo_gbt.cv_results_)
cross_validation_gbt['escenario']='modelo_gbt'

cross_validation_gbtt=pd.DataFrame(modelo_gbt_t.cv_results_)
cross_validation_gbtt['escenario']='modelo_gbtt'

cross_validation_rnn=pd.DataFrame(modelo_rnn.cv_results_)
cross_validation_rnn['escenario']='modelo_rnn'

cross_validation_rnnt=pd.DataFrame(modelo_rnn_t.cv_results_)
cross_validation_rnnt['escenario']='modelo_rnnt'

resultado_cross_total = pd.concat([cross_validation_rf,cross_validation_rft, cross_validation_gbt, cross_validation_gbtt,cross_validation_rnn,cross_validation_rnnt],
                            ignore_index = True)

columna = pd.DataFrame(resultado_cross_total['params'])

resultado_cross_total.drop(['params'],axis=1, inplace=True)
resultado_cross_total = resultado_cross_total.drop_duplicates()
resultado_cross_total.reset_index(inplace=True, drop=False)

In [ ]:
resultado_cross_total.to_csv('resultados_crossvalidacion_general.csv',sep=";",decimal=",")